# Project Euler

These are my solutions and notes on some of the early [Project Euler](projecteuler.net) problems.

According to the website, the first 100 problems may be spoiled, but none past 100.  I don't expect to get that far, so I don't think that should be a problem.

In [2]:
import math

## Problem 1

> If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23.
> 
> Find the sum of all the multiples of 3 or 5 below 1000.

This seems like a good opportunity to use the `sum` of a list comprehension.

In [3]:
sum([i for i in range(1000) if (i % 3 == 0 or i % 5 == 0)])

233168

## Problem 2

> Each new term in the Fibonacci sequence is generated by adding the previous two terms. By starting with 1 and 2, the first 10 terms will be:
> 
> 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, ...
> 
> By considering the terms in the Fibonacci sequence whose values do not exceed four million, find the sum of the even-valued terms.

I don't know how _pythonic_ this one is, but I think it's relatively simple to understand.

In [4]:
terms = [1, 2]
next_term = 3
while (next_term < 4_000_000):
    terms.append(next_term)
    next_term = terms[-1] + terms[-2]

sum([i for i in terms if i % 2 == 0])


4613732

## Problem 3

> The prime factors of 13195 are 5, 7, 13 and 29.
> 
> What is the largest prime factor of the number 600851475143 ?

Here, we're basically just going to build a [Seive of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes)

In [5]:
def prime_seive(n):
    """Returns the list of all primes less than or equal to n"""
    composite = [True] * n
    for i in range(2, n):
        if composite[i]:
            for v in range(2 * i, n, i):
                composite[v] = False
    return [i for i in range(2, n) if composite[i]]

target = 13195
prime_candidates = prime_seive(target)
[i for i in prime_candidates if target % i == 0]

[5, 7, 13, 29]

This works for the sample, but for the size of this problem, the above prime seive crashes after ~20 seconds.  Possibly due to too much memory.  We're going to need a better way.

This time, lets try building a factor generator.  This seems both more pythonic, less memory intensive, and much faster.

In [6]:
def factors(v):
    f = 2
    last_factor = -1
    while f * f <= v:
        if f != last_factor and v % f == 0:
            yield f
            last_factor = f
            v = v // f
            continue
        f += 1
    yield v

[i for i in factors(600851475143)][-1]

6857

## Problem 4

> A palindromic number reads the same both ways. The largest palindrome made from the product of two 2-digit numbers is 9009 = 91 × 99.
> 
> Find the largest palindrome made from the product of two 3-digit numbers.


Here, it seems like brute forcing a solution should still be quick:

In [7]:
def is_palindrome_number(num):
    return int(str(num)[::-1]) == num

largest = -1
for a in range(100,999):
    for b in range(100, 999):
        product = a * b
        if product > largest and is_palindrome_number(product):
            largest = product

largest

906609

That's great and all, but I think there's a more readable way of doing it:

In [8]:
palindrome_numbers = [
    a * b 
    for a in range(100, 999) 
    for b in range(100, 999) 
    if is_palindrome_number(a*b)
]
max(palindrome_numbers)


906609

## Problem 5

> 2520 is the smallest number that can be divided by each of the numbers from 1 to 10 without any remainder.
> 
> What is the smallest positive number that is evenly divisible by all of the numbers from 1 to 20?

We know that the result is going to need to be a multiple of 2520, since that's part of the example.  From there, we can get to the full solution quickly.

In [9]:
def evenly_divides(base, highest_divisor):
    return all([base % i == 0 for i in range(2,highest_divisor)])

i = 2520
while not evenly_divides(i, 20):
    i += 2520
i

232792560

This solution points to a more elegant solution:
step = 1
n = 2
Iterate up from n insteps of step until you find the first number divisible by n. [2]
Set the value you find to step, and inccrement n.

Lets try it:

In [10]:

def evenly_divides_upto(max):
    step = 1
    n, v = 1, 1
    while not n == max:
        n += 1
        while not v % n == 0:
            v += step
        step = v
    return v

evenly_divides_upto(20)

232792560

That seems like a more elegant solution!

For good measure, lets see what it looks like as a generator that provides all the steps, since I'm still trying to map my mind around building generators:

In [11]:

def evenly_divides_upto(max):
    step = 1
    n, v = 0, 1
    while not n == max:
        n += 1
        while not v % n == 0:
            v += step
        step = v
        yield v

[i for i in evenly_divides_upto(10)]

[1, 2, 6, 12, 60, 60, 420, 840, 2520, 2520]

I wasn't sure if there was a direct relationship between the items that require new items to be generated and primes, so I generated the first 25 and its prime status:

In [18]:
data = [
    [
        list(evenly_divides_upto(i))[-1], 
        f'{i}, which is not prime' if (len(list(factors(i))) > 1) else f'{i}, which is Prime',
        list(factors(i))
    ] for i in range(2,40)
]

data = [print(f'Result {a} is at index {b}. {c}') for a,b,c in data]


Result 2 is at index 2, which is Prime. [2]
Result 6 is at index 3, which is Prime. [3]
Result 12 is at index 4, which is not prime. [2, 2]
Result 60 is at index 5, which is Prime. [5]
Result 60 is at index 6, which is not prime. [2, 3]
Result 420 is at index 7, which is Prime. [7]
Result 840 is at index 8, which is not prime. [2, 4]
Result 2520 is at index 9, which is not prime. [3, 3]
Result 2520 is at index 10, which is not prime. [2, 5]
Result 27720 is at index 11, which is Prime. [11]
Result 27720 is at index 12, which is not prime. [2, 6]
Result 360360 is at index 13, which is Prime. [13]
Result 360360 is at index 14, which is not prime. [2, 7]
Result 360360 is at index 15, which is not prime. [3, 5]
Result 720720 is at index 16, which is not prime. [2, 8]
Result 12252240 is at index 17, which is Prime. [17]
Result 12252240 is at index 18, which is not prime. [2, 3, 3]
Result 232792560 is at index 19, which is Prime. [19]
Result 232792560 is at index 20, which is not prime. [2, 1

It turns out, the relationship isn't as obvious as I'd hoped.

## Problem 6

> The sum of the squares of the first ten natural numbers is,

$$1^2 + 2^2 + ... + 10^2 = 385$$

> The square of the sum of the first ten natural numbers is,

$$(1 + 2 + ... + 10)^2 = 55^2 = 3025$$

>Hence the difference between the sum of the squares of the first ten natural numbers and the square of the sum is $3025 - 385 = 2640$.
> 
>Find the difference between the sum of the squares of the first one hundred natural numbers and the square of the sum.

In [25]:
numbers = 100
sum_of_squares = sum([i**2 for i in range(numbers+1)])
square_of_susm = sum([i for i in range(numbers+1)])**2

square_of_susm - sum_of_squares

25164150

## Problem 7

> By listing the first six prime numbers: 2, 3, 5, 7, 11, and 13, we can see that the 6th prime is 13.
> 
> What is the 10 001st prime number?

In [44]:
def is_prime(n):
    return all(n % i for i in range(2, int(n ** 0.5) + 1))

def find_nth_prime(nth):
    last_prime = 1
    prime_i = 1
    test = 3
    while prime_i <= nth:
        test = last_prime + 1
        while not is_prime(test):
            test += 1
        last_prime = test
        prime_i += 1
    return last_prime

find_nth_prime(10_001)

104743

## Problem 8

> <p>The four adjacent digits in the 1000-digit number that have the greatest product are 9 × 9 × 8 × 9 = 5832.</p>

```
73167176531330624919225119674426574742355349194934
96983520312774506326239578318016984801869478851843
85861560789112949495459501737958331952853208805511
12540698747158523863050715693290963295227443043557
66896648950445244523161731856403098711121722383113
62229893423380308135336276614282806444486645238749
30358907296290491560440772390713810515859307960866
70172427121883998797908792274921901699720888093776
65727333001053367881220235421809751254540594752243
52584907711670556013604839586446706324415722155397
53697817977846174064955149290862569321978468622482
83972241375657056057490261407972968652414535100474
82166370484403199890008895243450658541227588666881
16427171479924442928230863465674813919123162824586
17866458359124566529476545682848912883142607690042
24219022671055626321111109370544217506941658960408
07198403850962455444362981230987879927244284909188
84580156166097919133875499200524063689912560717606
05886116467109405077541002256983155200055935729725
71636269561882670428252483600823257530420752963450
```

> Find the thirteen adjacent digits in the 1000-digit number that have the greatest product. What is the value of this product?

This seems doable as a brute force solution.

In [71]:
from functools import reduce
import operator

def mul(list):
    return reduce(operator.mul, list, 1)

def find_largest_mul(i, length):
    list = [int(char) for char in str(i)]
    return max(
            [mul(list[i:i+length]) for i in range(0, len(list) - length)]
        )

i = 7316717653133062491922511967442657474235534919493496983520312774506326239578318016984801869478851843858615607891129494954595017379583319528532088055111254069874715852386305071569329096329522744304355766896648950445244523161731856403098711121722383113622298934233803081353362766142828064444866452387493035890729629049156044077239071381051585930796086670172427121883998797908792274921901699720888093776657273330010533678812202354218097512545405947522435258490771167055601360483958644670632441572215539753697817977846174064955149290862569321978468622482839722413756570560574902614079729686524145351004748216637048440319989000889524345065854122758866688116427171479924442928230863465674813919123162824586178664583591245665294765456828489128831426076900422421902267105562632111110937054421750694165896040807198403850962455444362981230987879927244284909188845801561660979191338754992005240636899125607176060588611646710940507754100225698315520005593572972571636269561882670428252483600823257530420752963450
find_largest_mul(i, 13)

23514624000

## Problem 9

>A Pythagorean triplet is a set of three natural numbers, $a < b < c$, for which,
>
> $$a^2 + b^2 = c^2$$
>
> For example, $3^2 + 4^2 = 9 + 16 = 25 = 5^2$.
> 
> There exists exactly one Pythagorean triplet for which $a + b + c = 1000$.
>
>Find the product $abc$.

Another brute force search.

In [75]:
for a in range(500):
    for b in range(a+1, 500):
        for c in range(b + 1, 500):
            if (a+b+c == 1000) and (a**2 + b**2 == c**2):
                print(f'{a}^2 + {b}^2 = {c}^2')
                print(a*b*c)

200^2 + 375^2 = 425^2
31875000


## Problem 10

> The sum of the primes below 10 is 2 + 3 + 5 + 7 = 17.
> 
> Find the sum of all the primes below two million.

Well, we've essentially already solved this, and it's a perfect time to actually make use of our prime seive.

In [76]:
sum(prime_seive(2_000_000))

142913828922